<a href="https://colab.research.google.com/github/Sripathm2/UCLA_CS_245_Project5/blob/GNN/GNN/GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install spektral
import spektral
import numpy as np
import scipy.sparse as sp
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.data.loaders import SingleLoader
from spektral.layers import GCNConv

In [ ]:
% cd /content/drive/MyDrive/CS245/UCLA_CS_245_Project5-GNN/GNN
import build_data
Adj, X, y = build_data.build_data()

/content/drive/MyDrive/CS245/UCLA_CS_245_Project5-GNN/GNN


In [ ]:
print(Adj.shape)
print(X.shape)
print(y.shape)

(286, 52, 52)
(286, 52, 5)
(286, 52, 1)


In [ ]:
dataset = [spektral.data.Graph(x=np.random.randint(0,1,size=(52,5)), a=sp.csr_matrix((52,52)))] # Placeholder data

'''
The input can only be an adjacency matrix of shape node x node. Node features has the same issue.
'''
'''
x: np.array, the node features (shape (n_nodes, n_node_features));
a: np.array or scipy.sparse matrix, the adjacency matrix (shape (n_nodes, n_nodes));
e: np.array, the edge features (shape (n_nodes, n_nodes, n_edge_features) or (n_edges, n_edge_features));
'''

In [ ]:
# Parameters
channels = 16          # Number of channels in the first layer
dropout = 0.5          # Dropout rate for the features
l2_reg = 5e-4 / 2      # L2 regularization rate
learning_rate = 1e-2   # Learning rate
epochs = 200           # Number of training epochs
patience = 10          # Patience for early stopping
a_dtype = dataset[0].a.dtype  # Only needed for TF 2.1

N = 51          # Number of nodes in the graph
F = 5           # Original size of node features
y = 5           # Label

# GNN Class Definition
class GNN(Model):
  def __init__(self):
    super(Model,self).__init__()
    self.x_in = Input(shape=(F,))
    self.a_in = Input((N,), sparse=True, dtype=a_dtype)
    self.do_1 = Dropout(dropout)
    self.gc_1 = GCNConv(channels=channels,
                   activation='relu',
                   kernel_regularizer=l2(l2_reg),
                   use_bias=False
                  )
    self.do_2 = Droput(dropout)
    self.gc2 = GCNConv(channels=y,
                       activation='softmax'
                       use_bias=False
                       )
    def call(self, inputs):
      x_in = self.x_in(inputs[0])
      a_in = self.a_in(inputs[1])
      x = self.do_1(x_in)
      x = self.gc_1([x, a_in])
      x = self.do_2(x)
      return gc_2([x, a_in])




In [ ]:
# Parameters
channels = 16          # Number of channels in the first layer
dropout = 0.5          # Dropout rate for the features
l2_reg = 5e-4 / 2      # L2 regularization rate
learning_rate = 1e-2   # Learning rate
epochs = 200           # Number of training epochs
patience = 10          # Patience for early stopping
a_dtype = dataset[0].a.dtype  # Only needed for TF 2.1

N = 51          # Number of nodes in the graph
F = 5           # Original size of node features
y = 5           # Label


# Model definition
x_in = Input(shape=(F,))
a_in = Input((N,), sparse=True, dtype=a_dtype)

do_1 = Dropout(dropout)(x_in)
gc_1 = GCNConv(channels,
               activation='relu',
               kernel_regularizer=l2(l2_reg),
               use_bias=False)([do_1, a_in])
do_2 = Dropout(dropout)(gc_1)
gc_2 = GCNConv(y,
               activation='softmax',
               use_bias=False)([do_2, a_in])

# Build model
model = Model(inputs=[x_in, a_in], outputs=gc_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

# Train model
loader = SingleLoader(dataset)
model.fit(loader.load(),
          steps_per_epoch=loader.steps_per_epoch,
          epochs=epochs,
          callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)])
